In [1]:
!pip install "pdf2image" --user
!pip install "flask" --user
!pip install "PyMuPDF" --user

In [ ]:
# Cell 4 — Flask app and conversion function
import io, zipfile, uuid, os
from flask import Flask, request, send_file, render_template_string, jsonify, url_for
import fitz  # PyMuPDF
from PIL import Image
from werkzeug.utils import secure_filename

app = Flask(__name__)

# Config
ALLOWED_EXT = {'.pdf'}
MAX_PAGES = 50000                     # limit pages to avoid memory overload
OUTPUT_DIR = r"F:\PDF_Outputs"      # where converted zips will be saved
os.makedirs(OUTPUT_DIR, exist_ok=True)  # make sure folder exists

# Upload form with JS validation
INDEX_HTML = """
<!DOCTYPE html>
<html>
<head>
  <title>PDF2JPG Converter</title>
  <style>
    body { font-family: Arial, sans-serif;
        background: #f4f6f8; 
        display: flex; 
        justify-content: center; 
        align-items: center; 
        height: 100vh; 
        margin: 0; 
        color: #fff; }
    .container { 
        background: rgba(255, 255, 255, 0.95); 
        padding: 30px; 
        border-radius: 12px; 
        box-shadow: 0 8px 20px rgba(0,0,0,0.2); 
        width: 350px; 
        text-align: center; 
        color:#333; }
    h2 { margin-bottom: 20px; }
    input[type="file"], input[type="number"] { 
        margin: 10px 0; 
        padding: 8px; 
        width: 90%; 
        border: 1px solid #ccc; 
        border-radius: 6px; 
        font-size: 14px; }
    label { display: block; 
        text-align: left; 
        margin: 10px 0 5px; 
        font-weight: bold; }
    button { 
        margin-top: 15px; 
        padding: 10px 20px; 
        border: none; 
        border-radius: 6px; 
        background-color: #007BFF; 
        color: white; 
        font-size: 15px; 
        cursor: pointer; transition: 0.3s; }
    button:hover { background-color: #0056b3; }
    button:disabled { background-color: #aaa; cursor: not-allowed; }
    .download { margin-top: 20px; }
    .download a { text-decoration: none; background: #28a745; color: white; padding: 10px 18px; border-radius: 6px; }
    .download a:hover { background: #1e7e34; }
    .error { color: red; margin-top: 10px; }
  </style>
</head>
<body>
  <div class="container">
    <h2>PDF to JPG Converter</h2>
    <form id="pdfForm" action="/pdf-to-jpg" method="post" enctype="multipart/form-data">
      <label>Select PDF:</label>
      <input type="file" id="fileInput" name="file" accept=".pdf" required>

      <label>DPI:</label>
      <input type="number" name="dpi" value="150" min="50" max="400">

      <label>Quality:</label>
      <input type="number" name="quality" value="90" min="10" max="95">

      <button type="submit" id="convertBtn">Convert</button>
      <div class="error" id="errorMsg"></div>
    </form>
  </div>

<script>
const fileInput = document.getElementById('fileInput');
const convertBtn = document.getElementById('convertBtn');
const errorMsg = document.getElementById('errorMsg');

document.getElementById('pdfForm').addEventListener('submit', function(e) {
    if (!fileInput.files.length) {
        e.preventDefault(); // prevent form submission
        errorMsg.textContent = "No PDF file uploaded! Please select a PDF.";
    }
});
</script>
</body>
</html>
"""

def allowed(filename):
    parts = os.path.splitext(filename.lower())
    return len(parts) == 2 and parts[1] in ALLOWED_EXT

def pdf_to_images_bytes(pdf_bytes, dpi=150, quality=90, max_pages=MAX_PAGES):
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
    zoom = dpi / 72.0
    mat = fitz.Matrix(zoom, zoom)
    images = []
    try:
        page_count = min(doc.page_count, max_pages)
        for i in range(page_count):
            page = doc.load_page(i)
            pix = page.get_pixmap(matrix=mat, alpha=False)
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            b = io.BytesIO()
            img.save(b, format="JPEG", quality=int(quality))
            images.append(b.getvalue())
    finally:
        doc.close()
    return images

@app.route("/", methods=["GET"])
def home():
    return render_template_string(INDEX_HTML)

@app.route("/pdf-to-jpg", methods=["POST"])
def pdf_to_jpg_route():
    if 'file' not in request.files or request.files['file'].filename == '':
        return render_template_string(INDEX_HTML + '<div class="container"><p class="error">No PDF uploaded. Please select a PDF file!</p></div>')

    f = request.files['file']
    filename = secure_filename(f.filename)
    if not allowed(filename):
        return render_template_string(INDEX_HTML + '<div class="container"><p class="error">Invalid file type. Only PDF allowed.</p></div>')

    try:
        dpi = max(50, min(400, int(request.form.get('dpi', 150))))
    except:
        dpi = 150
    try:
        quality = max(10, min(95, int(request.form.get('quality', 90))))
    except:
        quality = 90

    pdf_bytes = f.read()
    try:
        imgs = pdf_to_images_bytes(pdf_bytes, dpi=dpi, quality=quality)
    except Exception as e:
        return render_template_string(INDEX_HTML + f'<div class="container"><p class="error">Conversion failed: {e}</p></div>')

    uid = uuid.uuid4().hex[:8]
    out_filename = f"{os.path.splitext(filename)[0]}_{uid}.zip"
    save_path = os.path.join(OUTPUT_DIR, out_filename)

    with zipfile.ZipFile(save_path, mode='w') as zf:
        for i, img_b in enumerate(imgs, start=1):
            zf.writestr(f"page_{i}.jpg", img_b)

    download_url = url_for("download_file", filename=out_filename)
    return render_template_string(INDEX_HTML + f"""
    <div class="container">
      <h2>Conversion Successful</h2>
      <p>Your file has been converted.</p>
      <div class="download">
        <a href="{download_url}">Download File</a>
      </div>
    </div>
    """)

@app.route("/download/<filename>")
def download_file(filename):
    file_path = os.path.join(OUTPUT_DIR, filename)
    if os.path.exists(file_path):
        return send_file(file_path, as_attachment=True)
    else:
        return "File not found!", 404

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Sep/2025 00:40:52] "POST /pdf-to-jpg HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2025 00:40:59] "POST /pdf-to-jpg HTTP/1.1" 200 -
